In [1]:
git config pull.rebase false  # merge

SyntaxError: invalid syntax (3474231917.py, line 1)

In [2]:
print ("Hello world")

Hello world


In [ ]:
# prompt: Import an excel file with a time column and 10 other columns called: Hip, Knee, Ankle, Heel (ventral), Instep (dorsal), Toe 1 (Proximal "Thumb"), Toe 2, Toe 3 (Middle), Toe 4, and Toe 5 (Distal "Pinky"). Within each of the 10 columns, there are 3 sub-columns called: x, y, and likelihood.

import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from datetime import datetime

df = pd.read_excel('file.xlsx')

# Extract time column
time_column = df['Time']

# Extract columns for body parts
hip_x = df['Hip x']
hip_y = df['Hip y']
hip_likelihood = df['Hip likelihood']

knee_x = df['Knee x']
knee_y = df['Knee y']
knee_likelihood = df['Knee likelihood']

# ... and so on for the remaining body parts

# Create a dictionary to store all body parts data
body_parts = {
    'Hip': {
        'x': hip_x,
        'y': hip_y,
        'likelihood': hip_likelihood
    },
    'Knee': {
        'x': knee_x,
        'y': knee_y,
        'likelihood': knee_likelihood
    },
    # ... and so on for the remaining body parts
}

In [ ]:
#git commit -m "first commit"